# 1.Setting up TigerGraph Cloud

In [ ]:
# Setup
!pip install -U pyTigerGraph
def pprint(string):
  print(json.dumps(string, indent=2))

In [ ]:
import pandas as pd
import json

# 2.Connecting to TG Cloud

In [ ]:
import pyTigerGraph as tg 

TG_SUBDOMAIN = 'tigergraph' 
TG_HOST = "https://" + TG_SUBDOMAIN + ".i.tgcloud.io" # GraphStudio Link

TG_USERNAME = "tigergraph"
TG_PASSWORD = "tigergraph"
TG_GRAPHNAME = "Demo"

conn = tg.TigerGraphConnection(host=TG_HOST, graphname=TG_GRAPHNAME, username=TG_USERNAME, password=TG_PASSWORD)
conn.apiToken = conn.getToken(conn.createSecret())

conn.gsql('ls')

'---- Graph Demo\nVertex Types:\n- VERTEX Video_Audio(PRIMARY_ID id INT, Title STRING, Youtube_Link STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"\n- VERTEX Transcript(PRIMARY_ID id INT, Preprocessed_Text SET<STRING>, Text STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"\n- VERTEX Sentiment(PRIMARY_ID id STRING, positive_sentiment FLOAT, negative_sentiment FLOAT, neutral_sentiment FLOAT, compound_sentiment FLOAT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"\n- VERTEX Keyword(PRIMARY_ID id STRING) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"\n- VERTEX Topic(PRIMARY_ID id INT) WITH STATS="OUTDEGREE_BY_EDGETYPE", PRIMARY_ID_AS_ATTRIBUTE="true"\nEdge Types:\n- DIRECTED EDGE has_transcript(FROM Video_Audio, TO Transcript) WITH REVERSE_EDGE="reverse_has_transcript"\n- DIRECTED EDGE reverse_has_transcript(FROM Transcript, TO Video_Audio) WITH REVERSE_EDGE="has_transcript"\n- DIRECTED EDGE has_sen

In [ ]:
df=pd.read_csv('/content/topics.csv')
df
df.columns=['video_id','dominant_topic','topic_keywords']
df.to_csv('topics.csv',index=False)
df

# 3.Flattening Syntax

### 3.1 Drop Flatten Query

In [ ]:
res = conn.gsql('''
    USE GRAPH Demo
    BEGIN
    DROP JOB load_flattened_topic_keywords
    END
    ''')
print(res)

### 3.2 Flatten Query

In [ ]:
#TO VERTEX Topic VALUES ($0, _),
#TO VERTEX Topic VALUES ($0, $1, _),
res = conn.gsql('''
    USE GRAPH Demo
    BEGIN
    CREATE LOADING JOB load_flattened_topic_keywords FOR GRAPH Demo {
    DEFINE FILENAME f;
    LOAD f
    TO TEMP_TABLE t1(dominant_topic,topic_keywords) VALUES ($1, flatten($2,"|",1)) USING QUOTE="double", SEPARATOR=",", HEADER="true", EOL="\\n";

    LOAD TEMP_TABLE t1
        TO VERTEX Keyword VALUES($"topic_keywords"),
        TO EDGE has_topic_keyword VALUES($"dominant_topic", $"topic_keywords");
    }
    END
    ''')
print(res)

### 3.3 Run Flatten Query

In [ ]:
# Load the topic_keywords_file with the 'load_topic_keywords' job
topic_keywords_file = '/content/topics.csv'
results = conn.uploadFile(topic_keywords_file, fileTag="f", jobName="load_flattened_topic_keywords")
print(json.dumps(results, indent=2))

# 4.Loading job for timestamp_list for a keyword

In [ ]:
import pandas as pd
df=pd.read_csv('/content/keywords_with_timestamps.csv')
df

### 4.1 Drop timestamp_list query

In [ ]:
res = conn.gsql('''
    USE GRAPH Demo
    BEGIN
    DROP JOB l1
    END
    ''')
print(res)

### 4.2 Run timestamp_list query

In [ ]:
results = conn.gsql('''
  USE GRAPH Demo
  BEGIN
  CREATE LOADING JOB l1 FOR GRAPH Demo {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Keyword VALUES($0) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  LOAD MyDataSource TO EDGE has_keyword VALUES($0, $1, SPLIT($2,"|")) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
  ''')
print(results)

In [ ]:
# Load the posts file wiht the 'load_posts' job
keyword_with_timestamps_file = '/content/keywords_with_timestamps.csv'
results = conn.uploadFile(keyword_with_timestamps_file, fileTag='MyDataSource', jobName='l1')
print(json.dumps(results, indent=2))

# 5.Loading Transcript set

In [ ]:
results = conn.gsql('''
  USE GRAPH Demo
  BEGIN
  CREATE LOADING JOB load_transcript_set FOR GRAPH Demo {
  DEFINE FILENAME MyDataSource;
  LOAD MyDataSource TO VERTEX Transcript VALUES($0, SPLIT($2, "|"), $3) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  LOAD MyDataSource TO EDGE has_transcript VALUES($0, $0) USING SEPARATOR=",", HEADER="true", EOL="\\n";
  }
  END
  ''')
print(results)

In [ ]:
transcript_set = '/content/out.csv'
results = conn.uploadFile(transcript_set, fileTag='MyDataSource', jobName='load_transcript_set')
print(json.dumps(results, indent=2))

# 6.Exploring Graph 

In [ ]:
results = conn.getVertexTypes()
print(f"Verticies: {results}")
vertices = results

results = conn.getEdgeTypes()
print(f"Edges: {results}")
edges = results

Verticies: ['Video_Audio', 'Transcript', 'Sentiment', 'Keyword', 'Topic']
Edges: ['has_transcript', 'has_sentiment', 'has_keyword', 'has_topic', 'has_topic_keyword']


In [ ]:

print(f"Results for Keyword vertex")
pprint(conn.getVertexType("Keyword"))

print("-----------------")
print(f"Results for has_keyword edge")
pprint(conn.getEdgeType("has_keyword"))


Results for Keyword vertex
{
  "Config": {
    "TAGGABLE": false,
    "STATS": "OUTDEGREE_BY_EDGETYPE",
    "PRIMARY_ID_AS_ATTRIBUTE": true
  },
  "IsLocal": true,
  "Attributes": [],
  "PrimaryId": {
    "AttributeType": {
      "Name": "STRING"
    },
    "IsPartOfCompositeKey": false,
    "PrimaryIdAsAttribute": true,
    "AttributeName": "id",
    "HasIndex": false,
    "internalAttribute": false,
    "IsPrimaryKey": false
  },
  "Name": "Keyword"
}
-----------------
Results for has_keyword edge
{
  "IsDirected": true,
  "ToVertexTypeName": "Keyword",
  "Config": {
    "REVERSE_EDGE": "reverse_has_keyword"
  },
  "IsLocal": true,
  "Attributes": [
    {
      "AttributeType": {
        "ValueTypeName": "STRING",
        "Name": "LIST"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "start_timestamp_list",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    }
  ],
  "FromVertexTypeName":

In [ ]:
print("-----------------")
print(f"Results for has_keyword edge")
pprint(conn.getEdgeType("has_keyword"))

-----------------
Results for has_keyword edge
{
  "IsDirected": true,
  "ToVertexTypeName": "Keyword",
  "Config": {
    "REVERSE_EDGE": "reverse_has_keyword"
  },
  "IsLocal": true,
  "Attributes": [
    {
      "AttributeType": {
        "ValueTypeName": "STRING",
        "Name": "LIST"
      },
      "IsPartOfCompositeKey": false,
      "PrimaryIdAsAttribute": false,
      "AttributeName": "start_timestamp_list",
      "HasIndex": false,
      "internalAttribute": false,
      "IsPrimaryKey": false
    }
  ],
  "FromVertexTypeName": "Video_Audio",
  "Name": "has_keyword"
}


### 6.1 Counting Data

In [ ]:
print("Vertex Counts")
for vertex in vertices:
  print(f"There are {conn.getVertexCount(vertex)} {vertex} vertices in the graph")

print("--------------")
print("Edge Counts")
for edge in edges:
  print(f"There are {conn.getEdgeCount(edge)} {edge} edges in the graph")

Vertex Counts
There are 26 Video_Audio vertices in the graph
There are 25 Transcript vertices in the graph
There are 25 Sentiment vertices in the graph
There are 708 Keyword vertices in the graph
There are 14 Topic vertices in the graph
--------------
Edge Counts
There are 25 has_transcript edges in the graph
There are 25 has_sentiment edges in the graph
There are 1941 has_keyword edges in the graph
There are 25 has_topic edges in the graph
There are 210 has_topic_keyword edges in the graph


# 7.Data Retrieval

### 7.1 Getting a Vertex

In [ ]:
results = conn.getVerticesById("Transcript", "1")
pprint(results)

### 7.2 Multiple Vertex

In [ ]:
tdf1 = conn.getVerticesById("Video_Audio", ["4","6"])
pprint(tdf1)

[
  {
    "v_id": "4",
    "v_type": "Video_Audio",
    "attributes": {
      "id": 4,
      "Title": "crypto_tax",
      "Youtube_Link": "https://www.youtube.com/embed/674-2Iv82W8"
    }
  },
  {
    "v_id": "6",
    "v_type": "Video_Audio",
    "attributes": {
      "id": 6,
      "Title": "digital_token",
      "Youtube_Link": "https://www.youtube.com/embed/lVrtPgymV7o"
    }
  }
]


### 7.3 Count Edges Connected to a Vertex

In [ ]:
results = conn.getEdgeCountFrom("Topic", "10")
pprint(results)

{
  "has_transcript": 0,
  "reverse_has_transcript": 0,
  "has_sentiment": 0,
  "reverse_has_sentiment": 0,
  "has_keyword": 0,
  "reverse_has_keyword": 0,
  "has_topic": 0,
  "reverse_has_topic": 6,
  "has_topic_keyword": 15,
  "reverse_has_topic_keyword": 0
}


### 7.4 Show all Edges Connected to a Vertex

In [ ]:
results = conn.getEdges("Keyword", "elephant")
pprint(results)
print("***** Total Edges for Video_Audio Vertex *******",len(results))

[
  {
    "e_type": "reverse_has_keyword",
    "directed": true,
    "from_id": "elephant",
    "from_type": "Keyword",
    "to_id": "14",
    "to_type": "Video_Audio",
    "attributes": {
      "start_timestamp_list": [
        "104.37"
      ]
    }
  },
  {
    "e_type": "reverse_has_keyword",
    "directed": true,
    "from_id": "elephant",
    "from_type": "Keyword",
    "to_id": "10",
    "to_type": "Video_Audio",
    "attributes": {
      "start_timestamp_list": [
        "29.01",
        "34.35",
        "53.73",
        "55.14",
        "61.05",
        "62.37",
        "65.04",
        "66.9",
        "149.52"
      ]
    }
  },
  {
    "e_type": "reverse_has_topic_keyword",
    "directed": true,
    "from_id": "elephant",
    "from_type": "Keyword",
    "to_id": "14",
    "to_type": "Topic",
    "attributes": {}
  },
  {
    "e_type": "reverse_has_topic_keyword",
    "directed": true,
    "from_id": "elephant",
    "from_type": "Keyword",
    "to_id": "10",
    "to_type": "T

In [ ]:
results = conn.getEdges("Topic", "1")
pprint(results)
print("***** Total Edges for Topic Vertex *******",len(results))

[
  {
    "e_type": "reverse_has_topic",
    "directed": true,
    "from_id": "1",
    "from_type": "Topic",
    "to_id": "16",
    "to_type": "Video_Audio",
    "attributes": {}
  },
  {
    "e_type": "reverse_has_topic",
    "directed": true,
    "from_id": "1",
    "from_type": "Topic",
    "to_id": "11",
    "to_type": "Video_Audio",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "1",
    "from_type": "Topic",
    "to_id": "mutual",
    "to_type": "Keyword",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "1",
    "from_type": "Topic",
    "to_id": "long",
    "to_type": "Keyword",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "1",
    "from_type": "Topic",
    "to_id": "small",
    "to_type": "Keyword",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "1",
    "from_

### 7.5 Convert Vertex/Edge Set to Dataframe

In [ ]:
df3 = conn.vertexSetToDataFrame(tdf1)
print(df3)

  v_id  id          Title                               Youtube_Link
0    4   4     crypto_tax  https://www.youtube.com/embed/674-2Iv82W8
1    6   6  digital_token  https://www.youtube.com/embed/lVrtPgymV7o


### 7.6 Get Edges

In [ ]:
df4 = conn.getEdgesDataframe("Keyword", "crypto")
print(df4)
df4.to_csv("Topic_3_Edges_Dataframe.csv")

  from_type from_id      to_type to_id  \
0   Keyword  crypto  Video_Audio     4   
1   Keyword  crypto  Video_Audio     3   
2   Keyword  crypto  Video_Audio     7   
3   Keyword  crypto  Video_Audio     5   
4   Keyword  crypto        Topic     8   
5   Keyword  crypto        Topic     3   
6   Keyword  crypto        Topic    19   

                                start_timestamp_list  
0  [13.95, 81.48, 85.29, 93.33, 123.21, 134.52, 2...  
1  [4.11, 10.17, 12.27, 26.64, 31.17, 43.14, 101....  
2                                    [37.44, 194.49]  
3        [3.45, 17.43, 68.94, 90.63, 118.92, 128.25]  
4                                                NaN  
5                                                NaN  
6                                                NaN  


# 8.CYTOSCAPE

In [ ]:
results = conn.getEdges("Video_Audio", "1")
topic_id = conn.getEdges("Video_Audio", "1")[-1]["to_id"]
print(topic_id)
results = conn.getEdges("Topic", "1")
#pprint(results)
print("***** Total Edges for Topic Vertex *******",len(results))

17
***** Total Edges for Topic Vertex ******* 17


In [ ]:
results = conn.runInstalledQuery("video_to_topic", params={"vid": "1"})
pprint(results)

[
  {
    "@@t_id": [
      17
    ]
  }
]


In [ ]:
res = conn.getEdges("Topic", '17')
pprint(res)

[
  {
    "e_type": "reverse_has_topic",
    "directed": true,
    "from_id": "17",
    "from_type": "Topic",
    "to_id": "1",
    "to_type": "Video_Audio",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "17",
    "from_type": "Topic",
    "to_id": "low",
    "to_type": "Keyword",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "17",
    "from_type": "Topic",
    "to_id": "point",
    "to_type": "Keyword",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "17",
    "from_type": "Topic",
    "to_id": "star",
    "to_type": "Keyword",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "17",
    "from_type": "Topic",
    "to_id": "long",
    "to_type": "Keyword",
    "attributes": {}
  },
  {
    "e_type": "has_topic_keyword",
    "directed": true,
    "from_id": "17",
    "from_

#  9.Path Finding
Find paths between vertices.

Supported are:
- shortestPath - one shortest path between vertices
- allPaths - all paths within the specified edge limit

In [ ]:
results = conn.shortestPath([("Video_Audio", "3")], [("Video_Audio", "4")])
pprint(results)

[{'type': 'Video_Audio', 'id': '3'}]
[{'type': 'Video_Audio', 'id': '4'}]
[
  {
    "vertices": [
      {
        "v_id": "coin",
        "v_type": "Keyword",
        "attributes": {
          "id": "coin"
        }
      },
      {
        "v_id": "3",
        "v_type": "Video_Audio",
        "attributes": {
          "id": 3,
          "Title": "crypto",
          "Youtube_Link": "https://www.youtube.com/embed/cHzkLgpzHUc"
        }
      },
      {
        "v_id": "4",
        "v_type": "Video_Audio",
        "attributes": {
          "id": 4,
          "Title": "crypto_tax",
          "Youtube_Link": "https://www.youtube.com/embed/674-2Iv82W8"
        }
      }
    ],
    "edges": [
      {
        "e_type": "reverse_has_keyword",
        "from_id": "coin",
        "from_type": "Keyword",
        "to_id": "4",
        "to_type": "Video_Audio",
        "directed": true,
        "attributes": {
          "start_timestamp_list": [
            "255.48"
          ]
        }
      },
  

# 10.Similarity Algo

Jaccard similarity-esque algorithm that will be referenced as a sub-query.

In [ ]:
results = conn.gsql('''
  USE GRAPH Demo
  CREATE QUERY similarity(Set<STRING> A, Set<STRING> B) FOR GRAPH Demo RETURNS (FLOAT){ 
	SetAccum<STRING> @@inter, @@uni;
	FLOAT similarity;
	
	IF A.size() != 0 AND B.size() !=0 THEN
	  @@inter = A INTERSECT B;
	  @@uni = A UNION B;
	
	  IF @@inter.size() == 0 THEN
	    similarity = 0;
	  ELSE 
	    similarity = @@inter.size()*1.0/@@uni.size();
	    END;
	ELSE
	  similarity = 0;
	  END;
	
	PRINT similarity;
	RETURN similarity;
}
  ''')
pprint(results)

"Using graph 'Demo'\nSemantic Check Fails: The query name similarity is used by another object! Please use a different name.\nFailed to create queries: [similarity]."


# 11.Queries

### 11.1 Query 1

In [ ]:
results = conn.gsql('''
  USE GRAPH Demo
  CREATE QUERY topic_keywords_from_Video_Audio(VERTEX<Video_Audio> vid) FOR GRAPH Demo SYNTAX v2 { 
  
  video = {vid};

  topic_keywords = SELECT k FROM video:va - (has_topic>) - Topic:t - (has_topic_keyword>) - Keyword:k;

  PRINT topic_keywords;
  }
  ''')
pprint(results)

"Using graph 'Demo'\nSemantic Check Fails: The query name topic_keywords_from_Video_Audio is used by another object! Please use a different name.\nFailed to create queries: [topic_keywords_from_Video_Audio]."


In [ ]:
results = conn.runInstalledQuery("topic_keywords_from_Video_Audio", params={"vid": "4"})
pprint(results)

[
  {
    "topic_keywords": [
      {
        "v_id": "india",
        "v_type": "Keyword",
        "attributes": {
          "id": "india"
        }
      },
      {
        "v_id": "crypto",
        "v_type": "Keyword",
        "attributes": {
          "id": "crypto"
        }
      },
      {
        "v_id": "long",
        "v_type": "Keyword",
        "attributes": {
          "id": "long"
        }
      },
      {
        "v_id": "study",
        "v_type": "Keyword",
        "attributes": {
          "id": "study"
        }
      },
      {
        "v_id": "make",
        "v_type": "Keyword",
        "attributes": {
          "id": "make"
        }
      },
      {
        "v_id": "cryptocurrency",
        "v_type": "Keyword",
        "attributes": {
          "id": "cryptocurrency"
        }
      },
      {
        "v_id": "thirty",
        "v_type": "Keyword",
        "attributes": {
          "id": "thirty"
        }
      },
      {
        "v_id": "high",
        "v_type":

### 11.2 Query 2

In [ ]:
results = conn.gsql('''
  USE GRAPH Demo
  CREATE QUERY keywords_with_timestamp_from_Video_Audio(VERTEX<Video_Audio> vid) FOR GRAPH Demo SYNTAX v2 { 
  
  video = {vid};

  keywords_with_timestamp = SELECT k FROM video:va - (has_keyword>some) - Keyword:k;
  

  PRINT keywords_with_timestamp;
  }
  ''')
pprint(results)

"Using graph 'Demo'\nline 7:66 no viable alternative at input 'from video:va - (has_keyword>some'\nline 7:72 missing ';' at '-'\nline 7:81 mismatched input ':' expecting {INTERSECT, MINUS, UNION, '%', '&', '+', '-', '*', '/', '.', ';', '>', '|', '<<'}\nParsing encountered 3 syntax error(s)\n\nFailed to create queries: [keywords_with_timestamp_from_Video_Audio]."


In [ ]:
conn.gsql('''
  USE GRAPH Demo
  INSTALL QUERY keywords_with_timestamp_from_Video_Audio
  ''')

"Using graph 'Demo'\nSemantic Check Fails: The query keywords_with_timestamp_from_Video_Audio cannot be found!\nQuery installation failed!"

In [ ]:
results = conn.runInstalledQuery("keywords_with_timestamp_from_Video_Audio", params={"vid": "4"})
pprint(results)

### 11.3 Query 3

In [ ]:
#Common Keywords
results = conn.gsql('''
  USE GRAPH Demo
  CREATE QUERY Common_Keywords(VERTEX<Video_Audio> vid) FOR GRAPH Demo {

    SetAccum<STRING> @@setA, @@setB, @@AintsctB;

    video = {vid};
    keywords_with_timestamps = SELECT k
        FROM video:v - (has_keyword>) - Keyword:k
        ACCUM @@setA += k.id;

    topic_keyword = SELECT k
        FROM video:v - (has_topic>) - Topic - (has_topic_keyword>) - Keyword:k
        ACCUM @@setB += k.id;
    
    @@AintsctB = @@setA INTERSECT @@setB;

  PRINT @@setA;
  PRINT @@setB;
  PRINT @@AintsctB;
  }
  ''')
pprint(results)

### 11.4 Query 4

In [ ]:
#Unique Keywords
results = conn.gsql('''
  USE GRAPH Demo
  CREATE QUERY Unique_Keywords(VERTEX<Video_Audio> vid) FOR GRAPH Demo {

    SetAccum<STRING> @@setA, @@setB, @@AminusB;

    video = {vid};
    keywords_with_timestamps = SELECT k
        FROM video:v - (has_keyword>) - Keyword:k
        ACCUM @@setA += k.id;

    topic_keyword = SELECT k
        FROM video:v - (has_topic>) - Topic - (has_topic_keyword>) - Keyword:k
        ACCUM @@setB += k.id;
    
    @@AminusB = @@setA MINUS @@setB;

  PRINT @@setA;
  PRINT @@setB;
  PRINT @@AminusB;
  }
  ''')
pprint(results)

### 11.5 Query 5: Version I



In [ ]:
#Compound Sentiment for each Video containg a particular keyword
results = conn.gsql('''
CREATE QUERY Keyword_Sentiment_1(VERTEX<Keyword> k) FOR GRAPH Demo {
    SetAccum<FLOAT> @@compoundSenti;
    keyword = {k};
    video_with_keyword = SELECT v
        FROM keyword:key - (<has_keyword) - Video_Audio:v;
  
    video_sentiment = SELECT s
        FROM video_with_keyword:v - (has_sentiment>) - Sentiment:s
        ACCUM @@compoundSenti += s.compound_sentiment;
    
  #PRINT video_with_keyword;
  #PRINT video_sentiment;
  PRINT @@compoundSenti;
  }
  ''')
pprint(results)

### 11.6 Query 5: Version II

In [ ]:
results = conn.gsql('''
CREATE QUERY Keyword_Sentiment_2(SET<VERTEX<Keyword>>k) FOR GRAPH Demo {
    SetAccum<FLOAT> @@compoundSent;
    keyword = {k};
    video_sentiment = SELECT s
        FROM keyword:key - (<has_keyword) - Video_Audio:v - (has_sentiment>) - Sentiment: s
        ACCUM @@compoundSent += s.compound_sentiment;
    
  #PRINT video_sentiment;
  #PRINT @@compoundSent;
  PRINT avg(@@compoundSent);
  }
  ''')
pprint(results)

In [ ]:
results = conn.runInstalledQuery("Keyword_Sentiment_2", params={'k':'small', 'k':'elephant'})#How to send set in the para
pprint(results)

### 11.7 Drop Query

In [ ]:
res = conn.gsql('''
    USE GRAPH Demo
    BEGIN
    DROP QUERY keywords_with_timestamp_from_Video_Audio
    END
    ''')
print(res)

#### Preprocessing

In [ ]:
df1=df.drop(df.columns[[2]],axis=1)

df1.to_csv('topic_vertex_data.csv',index=False)
df1

In [ ]:
df2=df.drop(df.columns[[0]],axis=1)
df2.to_csv('topic_keywords_vertex_data.csv',index=False)
df2

In [ ]:
lst=['99.99','12.34','34.45']
lst=list(map(float,lst))
lst=list(map(int,lst))
lst=sorted(lst)
lst

### 11.8 Query 6


# 12.Visualizations Page Dropdown

In [ ]:
tdf1 = conn.getVerticesById("Video_Audio", [str(i) for i in range (25)])

# pprint(tdf1)
df3 = conn.vertexSetToDataFrame(tdf1)
list1=df3['v_id']
list2=df3['Title']
# print(list1,list2)
output=df3.set_index('v_id').to_dict()['Title']
display(output)

{'0': 'astronomy',
 '1': 'white_dwarf',
 '10': 'elephant',
 '11': 'fd_mutual_finance',
 '12': 'finance',
 '13': 'jeffrey_archer',
 '14': 'mammoth',
 '15': 'murakami',
 '16': 'mutual_funds',
 '17': 'paulo_coelho',
 '18': 'penguin',
 '19': 'animal_farm',
 '2': 'chimp',
 '20': 'shakespeare',
 '21': 'snow_bunting',
 '22': 'super_nova',
 '23': 'vaccine_news',
 '24': 'whale',
 '3': 'crypto',
 '4': 'crypto_tax',
 '5': 'crypto_means',
 '6': 'digital_token',
 '7': 'cryptocurrency',
 '8': 'dodo',
 '9': 'dolphin'}